<h1 style = color:red => <b>Set up</b><h1>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q --upgrade google-cloud-translate==3.20.2 gensim tqdm regex


In [ ]:
SHARED = "/content/drive/Shareddrives/CS476"
model_path = f"{SHARED}/randomshuffle_5window_skipgram_300size.model"
cred_path  = f"{SHARED}/natural-potion-459221-v6-3032c41951d7.json"
ar2fr_pkl  = f"{SHARED}/google_ar2fr_single.pkl"
fr2ar_pkl  = f"{SHARED}/google_fr2ar_single.pkl"



In [ ]:
import os, random, time, pickle, regex
from tqdm import tqdm
from gensim.models import Word2Vec


In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = cred_path


In [ ]:
from google.cloud import translate_v2 as translate


<h1 style=color:red><b>Sample 1000 source words</b></h1>
<p>Sample 1 000 source words
You begin by randomly picking 1 000 words out of all the words your cross-lingual model knows.

Those become the “source” words you’ll test.</p>

In [ ]:
model = Word2Vec.load(model_path)


In [ ]:
vocabulary_list = vocabulary_list = model.wv.index_to_key


In [ ]:
# To performe sampling without replacement
from random import sample

# Seed 42 so it gives the same answer each time we run it
random.seed(42)
N_TEST = 1000

sample_all = sample(vocabulary_list,N_TEST)
print(sample_all)

['stations', 'تقدىر', 'كتاءب', 'déficit', 'factions', 'تجارىه', 'fmln', 'المنصه', 'بالغ', 'résolues', 'td', 'دولىه', 'تقىىم', 'النزاع', 'بتنظىم', 'juristes', 'dirige', 'النظرىه', 'ادرج', 'audacieuses', 'ضحاىا', 'بقاع', 'للجنسىن', 'كوت', 'mercenaire', 'القادمىن', 'بانتهاكات', 'social', 'النفس', 'gusmao', 'وتحت', 'courte', 'référendum', 'ىاذن', 'accédé', 'liés', 'préparation', 'فرىقها', 'واستمع', 'dépasse', 'irrégulières', 'بخاصه', 'risquait', 'لكى', 'بادماج', 'garant', 'définies', 'حىاتهم', 'pouvant', 'mérité', 'vance', 'concentrer', 'بغىر', 'صوت', 'indispensable', 'للدول', 'منسقه', 'والحق', 'finances', 'مونترىال', 'المعامله', 'contraintes', 'متهم', 'توغو', 'مخىمات', 'impôts', 'ومتابعتها', 'mal', 'peine', 'سباق', 'المستقبل', 'العنصر', 'الجاءر', 'fonde', 'وتعبءه', 'trésorerie', 'harare', 'وصىانتها', 'المعاصر', 'احدث', 'absolument', 'افضل', 'المستقل', 'تتعلق', 'مسلحون', 'الصك', 'جلساته', 'prévue', 'منهاج', 'inutile', 'تدعم', 'drapeaux', 'jugés', 'مساعدات', 'préside', 'بسرعه', 'analysant',

In [ ]:
def is_Arabic(word):
    re_arabic = regex.compile(r'\p{arabic}')
    return bool(re_arabic.search(word))


In [ ]:
ar_full = []
fr_full=[]

In [ ]:
for word in sample_all:
    if(is_Arabic(word)):
        ar_full.append(word)
    else:
        fr_full.append(word)

<h1 style=color:red><b>2. For each sampled word, find k cross‑lingual neighbours And get the accuracy</b></h1>

In [ ]:
client = translate.Client()


In [ ]:
def google_best(word, src_lang, tgt_lang):
    try:
        best = client.translate(
            word,
            source_language=src_lang,
            target_language=tgt_lang
        )["translatedText"].lower()
        return [best] if best.strip() else []
    except Exception as e:
        print(f"[warn] {word}: {e}")
        return []


In [ ]:
with open(ar2fr_pkl, "rb") as f:
    ar2fr = pickle.load(f)

with open(fr2ar_pkl, "rb") as f:
    fr2ar = pickle.load(f)


In [ ]:
"""ar2fr, fr2ar = {}, {}

for w in tqdm(ar_full, desc="AR→FR (single)"):
    ar2fr[w] = google_best(w, src_lang="ar", tgt_lang="fr")
    time.sleep(0.02)          # polite delay

for w in tqdm(fr_full, desc="FR→AR (single)"):
    fr2ar[w] = google_best(w, src_lang="fr", tgt_lang="ar")
    time.sleep(0.02)"""



'ar2fr, fr2ar = {}, {}\n\nfor w in tqdm(ar_full, desc="AR→FR (single)"):\n    ar2fr[w] = google_best(w, src_lang="ar", tgt_lang="fr")\n    time.sleep(0.02)          # polite delay\n\nfor w in tqdm(fr_full, desc="FR→AR (single)"):\n    fr2ar[w] = google_best(w, src_lang="fr", tgt_lang="ar")\n    time.sleep(0.02)'

In [ ]:
"""import pickle
with open("google_ar2fr_single.pkl", "wb") as f:
    pickle.dump(ar2fr, f)
with open("google_fr2ar_single.pkl", "wb") as f:
    pickle.dump(fr2ar, f)"""


'import pickle\nwith open("google_ar2fr_single.pkl", "wb") as f:\n    pickle.dump(ar2fr, f)\nwith open("google_fr2ar_single.pkl", "wb") as f:\n    pickle.dump(fr2ar, f)'

In [ ]:
k_values = [1,2,3,5,10]
scores = {k : 0 for k in k_values}

In [ ]:
for src in tqdm(sample_all, desc="Scoring"):
    neighbours = [v for v,_ in model.wv.most_similar(src, topn=50) if is_Arabic(v) != is_Arabic(src)]

    gold = ar2fr[src] if is_Arabic(src) else fr2ar[src]
    gold_set = set(gold)

    for k in k_values:
        if any(nn.lower() in gold_set for nn in neighbours[:k] ):
            scores[k] +=1



Scoring: 100%|██████████| 1000/1000 [00:04<00:00, 208.21it/s]


<h1 style=color:red><b>Find the accuracy</b></h1>


In [ ]:
print("Intrinsic words evaluation accuracy")
for k in k_values:
    acc = scores[k] / len(sample_all)
    print(f"  Top-{k:<2}: {acc:.2%}")


Intrinsic words evaluation accuracy
  Top-1 : 5.70%
  Top-2 : 6.40%
  Top-3 : 6.80%
  Top-5 : 8.20%
  Top-10: 9.00%
